In [ ]:
from huggingface_hub import login
import os

hf_token = os.getenv('HF_TOKEN')
login(hf_token)

In [ ]:
from IPython.display import HTML
iframe_html = """
<iframe src="https://huggingface.co/datasets/DIBT/MPEP_RUSSIAN/embed/viewer/train" width="80%" height="560px"></iframe>
"""
display(HTML(iframe_html))

In [ ]:
import pandas as pd
from datasets import load_dataset

dataset = load_dataset("DIBT/MPEP_RUSSIAN")
df = pd.DataFrame(dataset)

print(df.to_string(index=False))

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from distilabel.steps import ExpandColumns
from distilabel.llms import InferenceEndpointsLLM
from distilabel.pipeline import Pipeline
from distilabel.steps import LoadDataFromHub
from distilabel.steps.tasks import TextGeneration, UltraFeedback
from distilabel.steps import CombineColumns
from distilabel.steps import step
from distilabel.steps.typing import GeneratorStepOutput

llama70B = InferenceEndpointsLLM(
    model_id="meta-llama/Meta-Llama-3.1-70B-Instruct",
    tokenizer_id="meta-llama/Meta-Llama-3.1-70B-Instruct",
    generation_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.7
    }
)
llama405B = InferenceEndpointsLLM(
    model_id="meta-llama/Meta-Llama-3.1-405B-Instruct-FP8",
    tokenizer_id="meta-llama/Meta-Llama-3.1-405B-Instruct-FP8",
    generation_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.7
    }
)

with Pipeline(name="synthetic-data-with-llama3-russian-dibt") as pipeline:
    # load dataset with prompts
    load_dataset = LoadDataFromHub(
        repo_id="DIBT/MPEP_RUSSIAN",
        output_mappings={"target": "instruction"}
    )

    # generate two responses
    generate = [
        TextGeneration(
            llm=llama70B,
            output_mappings={"generation": "response"}
        ),
        TextGeneration(
            llm=llama405B,
            output_mappings={"generation": "response"}
        )
    ]

    # combine responses into one col
    combine = CombineColumns(
        columns=["response", "model_name"],
        output_columns=["responses", "model_names"]
    )

    # rate responses with 405B LLM-as-a-judge
    rate = UltraFeedback(
        aspect="overall-rating", 
        llm=llama405B, 
        input_mappings={"generations": "responses"}
    )

    # define and run pipeline
    load_dataset >> generate >> combine >> rate

In [ ]:
# distiset=pipeline.dry_run()
distiset=pipeline.run(use_cache=False)

In [ ]:
distiset['default']['train'].to_pandas()

In [ ]:
distiset.push_to_hub(
    "ZennyKenny/russian-dibt-llama-responses_new",
    token=hf_token,
    private=False
)

In [ ]:
iframe_html = """
<iframe src="https://huggingface.co/datasets/ZennyKenny/russian-dibt-llama-responses/embed/viewer/train" width="80%" height="560px"></iframe>
"""
display(HTML(iframe_html))